In [ ]:
#import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
df = pd.read_csv('SG.csv', sep=';', parse_dates = ['Time'])
df['Time'] = pd.to_datetime(df['Time'], utc=True)
df['Time'] = df['Time'].dt.tz_localize(None)
print(df.info())
df.set_index('Time', inplace=True)

df

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

- no grid backflow occurred - it could be dropped
- However, we could think of such a power balance equation:  
PV generation + Grid consumption + Battery discharging = Consumption + Battery charging + Grid backflow
  
- It can be simplified using (not taking losses into account for the sake of simplicity):  
Grid power = Grid consumption - Grid backflow  
Battery power = Battery charging - Battery discharging  
 - where ... power variables are Real numbers instead of R+  
  
- We get:  
PV generation + Grid power = Consumption + Battery power  
- we could extract Grid backflow even if it does not occur in the 'training' data (this system is probably not going to have Grid backflow anytime soon..)
- could be used for feature engineering and/or directly for modelling

In [ ]:
# just here for eda
df.drop(['Grid backflow'], axis=1, inplace=True)

In [ ]:
df[df['Consumption'].isna()]

In [ ]:
df.loc[pd.to_datetime('2022-03-17 23:00:00'):pd.to_datetime('2022-03-18 02:00:00'), :]

In [ ]:
df.loc[pd.to_datetime('2022-03-19 23:00:00'):pd.to_datetime('2022-03-20 02:00:00'), :]

- checked the approximate time frame of the missing data the day before and day after  
  -> battery charges after midnight
     - it might be caused by the optimization algorithm, selling everything at the end of the horizon to maximize profit/minimize loss 
     - check overall statistics if this is standard
     - check the balance between charging/discharging within 24h periods    
      
- decide how to fill NaNs later (overall strategy; not just this nb)

In [ ]:
df.loc[:, 'dayofweek'] = df.index.day_of_week
df.loc[:, 'date'] = df.index.date

df

In [ ]:
df.columns.to_list()

In [ ]:
# check missing rows
date_counts = df.groupby('date').count()['Consumption']
print(date_counts[date_counts != 48])
print("")

print("length of date_counts          : ", len(date_counts))
print("should be length of date counts: ", len(pd.date_range(start=date_counts.index[0], end=date_counts.index[-1], freq='D')))

# - no missing rows - from now on, I will assume that incoming csv files are not gonna have 'holes' in time series

In [ ]:
def stack_data(df: pd.DataFrame, window_length: int) -> np.array:
    stacked_data = []
    current_stack = []
    timestamps = df.index

    for i in range(len(timestamps)):
        current_stack.append(df.iloc[i].values)
        
        if len(current_stack) == window_length:
            stacked_data.append(np.array(current_stack))
            current_stack = []

    stacked_data = np.array(stacked_data)

    return stacked_data

In [ ]:
np_stacked_daily_data = stack_data(
    df.loc[pd.to_datetime('2022-03-06 00:00:00'):pd.to_datetime('2022-04-04 00:00:00'), :], 
    48
    )

print("shape of np_daily_data: ", np_stacked_daily_data.shape)
print("first day: ", np_stacked_daily_data[0, 0, -1])
print("last day : ", np_stacked_daily_data[-1, -1, -1])

# Visualization

In [ ]:
sns.pairplot(df.drop(['dayofweek', 'date'], axis=1), hue="PV generation", palette='viridis')

In [ ]:
sns.pairplot(df.drop(['dayofweek'], axis=1), hue="date", palette='viridis')

In [ ]:
sns.pairplot(df.drop(['date'], axis=1), hue="dayofweek", palette='viridis')

In [ ]:
plt.figure(figsize=(15, 12))
linestyles = ['--', '-.', ':']

for q in range(5):
    c = 0
    ax = plt.subplot(5, 1, q+1)
    
    for d in range(np_stacked_daily_data.shape[0]):
        if c < 10:
            ln = linestyles[0]
        elif c < 20:
            ln = linestyles[1]
        else:
            ln = linestyles[2]
        
        ax.plot(np.arange(np_stacked_daily_data.shape[1]), np_stacked_daily_data[d, :, q], linestyle=ln, label=f'{np_stacked_daily_data[d, 0, -1]}')
        ax.title.set_text(f'{df.columns[q]}')
        ax.set_xticks([*range(0, 50, 2)])
        ax.set_xticklabels([*range(0, 25, 1)])
        ax.grid()
        c+=1

plt.subplots_adjust(hspace=0.32)
plt.legend(bbox_to_anchor=(1.1, 6.3))
plt.show()

In [ ]:
# look closer at '2022-03-27', due to battery charging around 14:00 - 15:00

sel_idx = np.where(np_stacked_daily_data[:, 0, -1] == pd.to_datetime('2022-03-27').date())[0][0]

plt.figure(figsize=(15, 12))
for q in range(5):
    ax = plt.subplot(5, 1, q+1)
    ax.plot(np.arange(np_stacked_daily_data.shape[1]), np_stacked_daily_data[sel_idx, :, q], label=f'{np_stacked_daily_data[sel_idx, 0, -1]}')
    ax.title.set_text(f'{df.columns[q]}')
    ax.set_xticks([*range(0, 50, 2)])
    ax.set_xticklabels([*range(0, 25, 1)])
    ax.grid()

plt.subplots_adjust(hspace=0.32)
plt.legend(bbox_to_anchor=(1.1, 6.3))
plt.show()

- Battery charging and discharging overlap - these actions probably occured sequentially, and the shape of plots is caused by the relatively coarse granularity of timestamps 

In [ ]:
plt.figure(figsize=(15, 12))
for q in range(5):
    ax = plt.subplot(5, 1, q+1)
    ax.boxplot(np_stacked_daily_data[:, :, q])
    ax.title.set_text(f'{df.columns[q]}')
    ax.set_xticks([*range(0, 50, 2)])
    ax.set_xticklabels([*range(0, 25, 1)])
    ax.yaxis.grid()

plt.subplots_adjust(hspace=0.32)
plt.show()

- Consumption and Grid Consumption show increase min-max spread in the mornings and afternoons - because of UTC timestamp; human activity depends on local time
- If local time (CET/CEST) would be still used, such effects would be observable on PV generation
- keep using UTC + try bool variable like 'summer_time' using .dt.utcoffset()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(15, 10))

for d in range(np_stacked_daily_data.shape[0]):
    if np_stacked_daily_data[d, 10, -2] in [*range(5)]:
        ax1.scatter(np_stacked_daily_data[d, :, 2], np_stacked_daily_data[d, :, 0], color='b')
        ax3.scatter(np_stacked_daily_data[d, :, 2], np_stacked_daily_data[d, :, 1], color='orange')
    else:
        ax2.scatter(np_stacked_daily_data[d, :, 2], np_stacked_daily_data[d, :, 0], color='g')
        ax4.scatter(np_stacked_daily_data[d, :, 2], np_stacked_daily_data[d, :, 1], color='r')        

ax1.title.set_text("work days")
ax1.set_ylabel("Consumption")
ax1.set_zorder(1)
ax1.grid()

ax3.set_xlabel("PV generation")
ax3.set_ylabel("Grid Consumption")
ax3.grid()

ax2.title.set_text("weekends")
ax2.set_ylabel("Consumption")
ax2.grid()

ax4.set_xlabel("PV generation")
ax4.set_ylabel("Grid Consumption")
ax4.grid()

plt.subplots_adjust(hspace=0.02, wspace=0.17)
plt.show()

In [ ]:
# overall correlation matrix
df_corr = df.iloc[:, :5].corr()

# heatmap
plt.figure(figsize = (5,4))
sns.heatmap(df_corr, 
        xticklabels=df_corr.columns,
        yticklabels=df_corr.columns, 
        center=0)
plt.title('Correlation matrix')
plt.show()